# Official Results for 'Discovering Foodborne Illness in Online Restaurant Reviews' in JAMIA, 2017

This notebook presents the official evaluation of classifiers presented in the paper.

Because of licensing restrictions, we cannot publish the data used in the experiments.

However, the final trained models may be found in the `best_models` directory for inspection and further experimentation.

Additionally, those models are all retrained on the full training data within this notebook with a fixed random seed so that the final numbers could be reproduced if needed.

For inqueries about the results, models, or data, please contact [Tom Effland](mailto:teffland@cs.columbia.edu).

## Table of Contents

1. [Derivations of Selection Bias Corrections](#Selection-Bias-Correction)
    1. [Training](#Training:-Error-Rate)
    2. [Testing](#Testing)
    
2. [Setup](#Setup)
    1. [Data Ingestion](#Data-Ingestion) (Data is cleaned ahead of time)
    4. [Loading Models](#Loading-Models) (Model hyperparams are pre-tuned)
3. [Sick Task](#Sick-Task)
    1. [Logistic Regression](#Sick:-Logistic-Regression)
        1. [Precision Recall Tradeoffs](#Precision-Recall-Tradeoffs)
    2. [Random Forest](#Sick:-Random-Forest)
    3. [SVM](#Sick:-SVM)
    4. [Prototype](#Sick:-Prototype)
4. [Multiple Task](#Multiple-Task)
    1. [Logistic Regression](#Multiple:-Logistic-Regression)
    2. [Random Forest](#Multiple:-Random-Forest)
    3. [SVM](#Multiple:-SVM)
    4. [Pipelined Logistic Regression](#Multiple:-Pipelined-Logistic-Regression)
    5. [Prototype](#Multiple:-Prototype)

# Selection Bias Correction

Let $T(x)$ be the biased selection process for labelling data at DOHMH. We treat it as a black-box and model it atomically.

Let $U$ be the set of all Yelp Reviews that have been processed by the system.

Let $B \subset U$ s.t. $B = \{(x,y) | T(x) = 1\}$. $B$ is the biased training set.

Let $B^c \subseteq U \setminus B = \{(x,y) | T(x) = 0\}$. $B^c$ is the biased complement training set.

## Training: Error Rate

We can model the error rate of some classifier $f$ as:

$p(f(x) \ne y) = p(f(x) \ne y | T(x) = 1)p(T(x) = 1) + p(f(x) \ne y | T(x) = 0)p(T(x) = 0)$

and use plugin estimates:

$\hat{p}(T(x) = 1) = \frac{|B|}{|U|}$ (the % of labeled points)

$\hat{p}(f(x) \neq y | T(x)=1) = \frac{1}{|B|}\Sigma_{(x,y) \in B}{I[f(x) \neq y]}$

likewise,

$\hat{p}(f(x) \neq y | T(x)=0) = \frac{1}{|B^c|}\Sigma_{(x,y) \in B^c}{I[f(x) \neq y]}$

therefore

$\hat{p}(f(x) \neq y) = w_B\Sigma_{(x,y) \in B}{I[f(x) \neq y]} + w_{B^c}\Sigma_{(x,y) \in B^c}{I[f(x) \neq y]}$

where 

$w_B = \frac{1}{|U|}$ and $w_{B^c} = (1 - \frac{|B|}{|U|})(\frac{1}{|B^c|})$

Note that when $B^c$ is the entire complement of $B$, then $w_{B^c}$ reduces to $\frac{1}{|U|}$

So, if we are trying to minimize the classification error in the objective function then these are the sample weights to be used when calculating the error

## Testing

We will evaluate models using the F1 score and AUPR metric, calculated empirically on the dev and test sets.

We can bias-correct these the same way, using bias corrections for the precision and recall. Consider the probabilistic form of precision for some classifier $f$:

#### Precision

Let $U$ be the test set.

$Precision = p(y=1|f(x)=1) = \Sigma_{t}{p(y=1 | f(x)=1, T(x) = t)p(T(x)=t|f(x)=1)}$

Let $U_P \subset U$ be the set $\{(x,y) | f(x) = 1\}$. 

**Note** that $U_P$ varies with the classification threshold.

Then as with the error rate, we can find plugin estimates for the necessary quanities.

$\hat{p}(T(x) = 1|f(x)=1) = \frac{1}{|U_P|} \Sigma_{(x,y) \in B}{I[f(x)=1]} = \frac{|B \cap U_P|}{|U_P|}$

and so $\hat{p}(T(x) = 0|f(x)=1) = 1 - \frac{|B \cap U_P|}{|U_P|}$

Also,

$\hat{p}(y=1|f(x)=1, T(x)=1) = \frac{1}{|U_P|} \Sigma_{(x,y) \in B\cap U_P}{I[y=1]} 
 $

and $\hat{p}(y=1|f(x)=1, T(x)=0) = \frac{1}{|U_P|} \Sigma_{(x,y) \in B^c\cap U_P}{I[y=1]}$

Therefore

$Precision = \frac{|B \cap U_P|}{|U_P|} \frac{1}{|U_P|} \Sigma_{(x,y) \in B\cap U_P}{I[y=1]} 
+ (1 - \frac{|B \cap U_P|}{|U_P|})\frac{1}{|U_P|} \Sigma_{(x,y) \in B^c\cap U_P}{I[y=1]}$

#### Recall

The derivation for improtance-weighted (IW) recall follows similary to IW precision.

$Recall = p(f(x)=1|y=1) = \Sigma_{t}{p(f(x)=1| y=1, T(x) = t)p(T(x)=t|y=1)}$

Let $U_R \subset U$ be the set $\{(x,y) | y = 1\}$. 

**Note** that $U_R$ **does not** vary with the classification threshold.

The plugin estimates are:

$\hat{p}(T(x) = 1|y=1) = \frac{1}{|U_R|} \Sigma_{(x,y) \in B}{I[y=1]} = \frac{|B \cap U_R|}{|U_R|}$

and so $\hat{p}(T(x) = 0|y=1) = 1 - \frac{|B \cap U_R|}{|U_R|}$

Also,

$\hat{p}(f(x)=1 |y=1, T(x)=1) = \frac{1}{|U_R|} \Sigma_{(x,y) \in B\cap U_R}{I[f(x)=1]}$

and $\hat{p}(f(x)=1|y=1, T(x)=0) = \frac{1}{|U_R|} \Sigma_{(x,y) \in B^c\cap U_R}{I[f(x)=1]}$

plugging in we have

$ Recall =  \frac{|B \cap U_R|}{|U_R|} \frac{1}{|U_R|} \Sigma_{(x,y) \in B\cap U_R}{I[f(x)=1]}
+ (1 - \frac{|B \cap U_R|}{|U_R|}) \frac{1}{|U_R|} \Sigma_{(x,y) \in B^c\cap U_R}{I[f(x)=1]}$

#### AUPR Curve

Finally, using the above plugin estimates, we can balance data from both $B$ and (a sample from) $B^c$.  We can obtain a series of ordered Precision-Recall points $E = \{(p,r,t)_i | r_i \leq r_{i'}\}$ by varying the classification threshold $t \in [0,1]$ and then using trapezoidal integration to approximate the area under the Recall vs. Precision curve.

#### Bootstrap

For the final evaluation, we would like confidence intervals about the F1 and AUPR. We find these by using the percentile bootstrap:

We calculate bootstrap statistics for the IW-Precision, IW-Recall, and IW-AUPR as follows:

First we calculate the statistic $\bar{x}$ (for each of IW-Precision, IW-Recall, and IW-AUPR).

Then we resample the test dataset with replacement $B$ times and obtain the bootstrap statisic estimates for each set.

Call these $x_1, ...,  x_B$.

Then we can compute confidence intervals around $\bar{x}$ the usual way by finding the $\alpha=.025$ boundary quantiles $\delta_{\alpha}, \delta_{1-\alpha}$, such that 

$$P(\bar{x}^* - \delta_{1-\alpha} \leq \bar{x} \leq \bar{x}^* - \delta_{\alpha}) = .95$$

# Setup

In [1]:
import time
%matplotlib inline
from copy import deepcopy
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')
from __future__ import print_function

In [2]:
from experiments.baseline_experiment_util import *

In [3]:
random_seed = 0

## Data Ingestion

In [4]:
biased_data = setup_baseline_data(data_path='data', train_regime='biased', test_regime='gold', random_seed=random_seed)
gold_data = setup_baseline_data(data_path='data', train_regime='gold', test_regime='gold', random_seed=random_seed)
silver_data = setup_baseline_data(data_path='data', train_regime='silver', test_regime='gold', random_seed=random_seed)

In [5]:
# train setup
biased_text = np.array(biased_data['train_data']['text'])
biased_sick = np.array(biased_data['train_data']['is_foodborne'])
biased_mult = np.array(biased_data['train_data']['is_multiple'])
biased_weights = calc_train_importance_weights(biased_data['train_data']['is_biased'], biased_data['U'])

gold_text = np.array(gold_data['train_data']['text'])
gold_sick = np.array(gold_data['train_data']['is_foodborne'])
gold_mult = np.array(gold_data['train_data']['is_multiple'])
gold_weights = calc_train_importance_weights(gold_data['train_data']['is_biased'], gold_data['U'])

silver_text = np.array(silver_data['train_data']['text'])
silver_sick = np.array(silver_data['train_data']['is_foodborne'])
silver_mult = np.array(silver_data['train_data']['is_multiple'])
silver_weights = calc_train_importance_weights(silver_data['train_data']['is_biased'], silver_data['U'])

# test setup
test_data = gold_data['test_data']
B = 1000 # number of bootstrap test set resamples
all_results = {}

## Loading Models

In [6]:
from experiments.lr_model import model as lr_model
from experiments.rf_model import model as rf_model
from experiments.svm_model import model as svm_model
from sklearn.externals import joblib

In [7]:
# logistic regression sick models
best_sick_lr_biased = joblib.load('experiments/best_models/best_lr_sick_biased.pkl')
best_sick_lr_gold = joblib.load('experiments/best_models/best_lr_sick_gold.pkl')
best_sick_lr_silver = joblib.load('experiments/best_models/best_lr_sick_silver.pkl')

In [8]:
# random forest sick models
best_sick_rf_biased = joblib.load('experiments/best_models/best_rf_sick_biased.pkl')
best_sick_rf_gold = joblib.load('experiments/best_models/best_rf_sick_gold.pkl')
best_sick_rf_silver = joblib.load('experiments/best_models/best_rf_sick_silver.pkl')

In [9]:
# svm sick models
best_sick_svm_biased = joblib.load('experiments/best_models/best_svm_sick_biased.pkl')
best_sick_svm_gold = joblib.load('experiments/best_models/best_svm_sick_gold.pkl')
best_sick_svm_silver = joblib.load('experiments/best_models/best_svm_sick_silver.pkl')

In [10]:
# logistic regression multiple models
best_mult_lr_biased = joblib.load('experiments/best_models/best_lr_mult_biased.pkl')
best_mult_lr_gold = joblib.load('experiments/best_models/best_lr_mult_gold.pkl')
best_mult_lr_silver = joblib.load('experiments/best_models/best_lr_mult_silver.pkl')

In [11]:
# random forest multiple models
best_mult_rf_biased = joblib.load('experiments/best_models/best_rf_mult_biased.pkl')
best_mult_rf_gold = joblib.load('experiments/best_models/best_rf_mult_gold.pkl')
best_mult_rf_silver = joblib.load('experiments/best_models/best_rf_mult_silver.pkl')

In [12]:
# svm multiple models
best_mult_svm_biased = joblib.load('experiments/best_models/best_svm_mult_biased.pkl')
best_mult_svm_gold = joblib.load('experiments/best_models/best_svm_mult_gold.pkl')
best_mult_svm_silver = joblib.load('experiments/best_models/best_svm_mult_silver.pkl')

## Sick Task

Here are the notable stats for testing on the Sick task:
* All the test data is from 1/1/2017 and later
* It's about 2/3 biased and 1/3 nonbiased (1975 and 1000 reviews, respectively)
* All 1000 nonbiased reviews are have `No` labels
* The 1975 biased reviews are about 52% `Yes`/`No` (1026/949)

### Sick: Logistic Regression

In [13]:
best_sick_lr_biased.fit(biased_text, biased_sick, logreg__sample_weight=biased_weights)
best_sick_lr_gold.fit(gold_text, gold_sick, logreg__sample_weight=gold_weights)
best_sick_lr_silver.fit(silver_text, silver_sick, logreg__sample_weight=silver_weights)
print()

In [ ]:
start = time.time()
title = 'Logistic Regression Sick Biased'
all_results[title] = model_report(best_sick_lr_biased, title, 'is_foodborne',
                                  test_data=test_data,
                                  save_fname='figures/sick_lr_biased',
                                  B=B, random_seed=random_seed)
print('\n{} seconds for evaluation'.format(int(time.time()-start)))


B: 122/1000 

In [ ]:
start = time.time()
title = 'Logistic Regression Sick Gold'
all_results[title] = model_report(best_sick_lr_gold, title, 'is_foodborne', 
                                  test_data=test_data,
                                   save_fname='figures/sick_lr_gold',
                                   B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'Logistic Regression Sick Silver'
all_results[title] = model_report(best_sick_lr_silver, title, 'is_foodborne', 
                                  test_data=test_data,
                                  save_fname='figures/sick_lr_silver',
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

### Precision Recall Tradeoffs

We'd like to explore how we lose precision with the LR models, as we gain recall. This can be visualized by looking at the high recall region of the PR curves. 

In the curve we can see that all of the model precision begins to drop around a recall of .8 start to significantly drop precision around a recall of .9

In [ ]:
pr_curves([best_sick_lr_biased, best_sick_lr_gold, best_sick_lr_silver], 
          ['"Sick" Logistic Regression Biased', 
           '"Sick" Logistic Regression Gold', 
           '"Sick" Logistic Regression Silver'], 
          'Precision-Recall Tradeoffs', 'is_foodborne', 
          dashes=[[20,5], [10,3], [5,1]],
          test_data=test_data, save_fname='figures/paper_sick_lr',
          figsize=(6,4),
          xlim=(.5,1.),
          yticks=.1*np.arange(11))

In [ ]:
print_model_hyperparams(best_sick_lr_biased, 'Logistic Regression Sick Biased')
print()
print_model_hyperparams(best_sick_lr_gold, 'Logistic Regression Sick Gold')
print()
print_model_hyperparams(best_sick_lr_silver, 'Logistic Regression Sick Silver')
print()

### Sick: Random Forest

In [ ]:
best_sick_rf_biased.fit(biased_text, biased_sick, rf__sample_weight=biased_weights)
best_sick_rf_gold.fit(gold_text, gold_sick, rf__sample_weight=gold_weights)
best_sick_rf_silver.fit(silver_text, silver_sick, rf__sample_weight=silver_weights)
print()

In [ ]:
start = time.time()
title = 'Random Forest Sick Biased'
all_results[title] = model_report(best_sick_rf_biased, title, 'is_foodborne',
                                  save_fname='figures/sick_rf_biased',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'Random Forest Sick Gold'
all_results[title] = model_report(best_sick_rf_gold, title, 'is_foodborne',
                                  save_fname='figures/sick_rf_gold',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'Random Forest Sick Silver'
all_results[title] = model_report(best_sick_rf_silver, title, 'is_foodborne',
                                  save_fname='figures/sick_rf_silver',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
print_model_hyperparams(best_sick_rf_biased, 'Random Forest Sick Biased')
print()
print_model_hyperparams(best_sick_rf_gold, 'Random Forest Sick Gold')
print()
print_model_hyperparams(best_sick_rf_silver, 'Random Forest Sick Silver')
print()

### Sick: SVM

In [ ]:
best_sick_svm_biased.fit(biased_text, biased_sick, svc__sample_weight=biased_weights)
best_sick_svm_gold.fit(gold_text, gold_sick, svc__sample_weight=gold_weights)
best_sick_svm_silver.fit(silver_text, silver_sick, svc__sample_weight=silver_weights)
print()

In [ ]:
start = time.time()
title = 'SVM Sick Biased'
all_results[title] = model_report(best_sick_svm_biased, title, 'is_foodborne',
                                  save_fname='figures/sick_svm_biased',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'SVM Sick Gold'
all_results[title] = model_report(best_sick_svm_gold, title, 'is_foodborne',
                                  save_fname='figures/sick_svm_gold',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'SVM Sick Silver'
all_results[title] = model_report(best_sick_svm_silver, title, 'is_foodborne',
                                  save_fname='figures/sick_svm_silver',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
print_model_hyperparams(best_sick_svm_biased, 'SVM Sick Biased')
print()
print_model_hyperparams(best_sick_svm_gold, 'SVM Sick Gold')
print()
print_model_hyperparams(best_sick_svm_silver, 'SVM Sick Silver')
print()

### Sick: Prototype

The prototype classifier is build in the Weka library in Java. To evaluate this, we ran the model on the same test data in the java environment and output the scores to csv.

In [ ]:
prototype_sick_scores = pd.read_csv('data/prototype_sick_scores.csv')
prototype_sick_scores.head()

In [ ]:
prototype_sick_results = prototype_model_report(test_data['is_foodborne'], 
                                    prototype_sick_scores['0'].tolist(), 
                                    test_data['is_biased'], 'J4.8', 
                                    B=B, random_seed=random_seed)
all_results['Prototype Sick'] = prototype_sick_results

# Multiple Task

Here are the notable stats for testing on the Multple task:
* All the test data is from 1/1/2017 and later
* It's about 2/3 biased and 1/3 nonbiased (1975 and 1000 reviews, respectively)
* All 1000 nonbiased reviews all have `No` labels
* The 1975 biased reviews are about 14% `Yes` (277 `Yes`, 1698 `No`)

### Multiple: Logistic Regression

In [ ]:
best_mult_lr_biased.fit(biased_text, biased_mult, logreg__sample_weight=biased_weights)
best_mult_lr_gold.fit(gold_text, gold_mult, logreg__sample_weight=gold_weights)
best_mult_lr_silver.fit(silver_text, silver_mult, logreg__sample_weight=silver_weights)
print()

In [ ]:
start = time.time()
title = 'Logistic Regression Mult Biased'
all_results[title] = model_report(best_mult_lr_biased, title, 'is_multiple',
                                  save_fname='figures/mult_lr_biased',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'Logistic Regression Mult Gold'
all_results[title] = model_report(best_mult_lr_gold, title, 'is_multiple',
                                  save_fname='figures/mult_lr_gold',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'Logistic Regression Mult Silver'
all_results[title] = model_report(best_mult_lr_silver, title, 'is_multiple',
                                  save_fname='figures/mult_lr_silver',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('{} seconds'.format(int(time.time()-start)))

In [ ]:
print_model_hyperparams(best_mult_lr_biased, 'Logistic Regression Mult Biased')
print()
print_model_hyperparams(best_mult_lr_gold, 'Logistic Regression Mult Gold')
print()
print_model_hyperparams(best_mult_lr_silver, 'Logistic Regression Mult Silver')
print()

### Multiple: Pipelined Logistic Regression

Since the Multiple is always conditioned on Sick=Yes (by definition), it makes more sense to train and use the Multuple model only on data where Sick=Yes. This may allow the model to focus better on Multiple-related features and not need to also learn the Sick class.

We try this technique for all three training regimes.

In [ ]:
class MultPipeline():
    def __init__(self, sick_model, mult_model, sick_threshold=.5):
        self.sick_model = sick_model
        self.mult_model = mult_model
        self.t = sick_threshold
    
    def predict_proba(self, xs):
        sick_probs = self.sick_model.predict_proba(xs)
        not_sick = sick_probs[:,1] < self.t
        mult_probs = self.mult_model.predict_proba(xs)
        mult_probs[not_sick] = sick_probs[not_sick]
        return mult_probs
    
    def predict(self, xs):
        return (self.predict_proba(xs)[:,1] >= self.t).astype(np.int32)

In [ ]:
sick_only = biased_sick.astype(np.bool)
sick_only_text = biased_text[sick_only]
sick_only_mult = biased_mult[sick_only]
sick_only_weights = biased_weights[sick_only]
print("{} training data / {} orginally".format(len(sick_only_text), len(biased_text)))
print("{0:2.2f}% yes, {1:2.2f}% biased".format(
    100.*sick_only_mult.sum()/len(sick_only_mult), 
    100.*np.array(biased_data['train_data']['is_biased'])[sick_only].sum()/len(sick_only_mult), 
))

#### Biased

In [ ]:
sick_only_mult_lr_biased = deepcopy(best_mult_lr_biased)

In [ ]:
sick_only_mult_lr_biased.fit(sick_only_text, sick_only_mult, logreg__sample_weight=sick_only_weights)
sick_only_mult_model_biased = MultPipeline(best_sick_lr_biased, sick_only_mult_lr_biased)

In [ ]:
title = 'Logistic Regression - Sick Only Biased Multiple'
all_results[title] = model_report(sick_only_mult_model_biased, title, 'is_multiple', 
                                  B=B, random_seed=random_seed)

#### Gold

In [ ]:
sick_only_mult_lr_gold = deepcopy(best_mult_lr_gold)

In [ ]:
sick_only_mult_lr_gold.fit(sick_only_text, sick_only_mult, logreg__sample_weight=sick_only_weights)
sick_only_mult_model_gold = MultPipeline(best_sick_lr_gold, sick_only_mult_lr_gold)

In [ ]:
title = 'Logistic Regression - Sick Only Gold Multiple'
all_results[title] = model_report(sick_only_mult_model_gold, title, 'is_multiple', 
                                  B=B, random_seed=random_seed)

#### Silver

In [ ]:
sick_only_mult_lr_silver = deepcopy(best_mult_lr_silver)

In [ ]:
sick_only_mult_lr_silver.fit(sick_only_text, sick_only_mult, logreg__sample_weight=sick_only_weights)
sick_only_mult_model_silver = MultPipeline(best_sick_lr_silver, sick_only_mult_lr_silver)

In [ ]:
title = 'Logistic Regression - Sick Only Silver Multiple'
all_results[title] = model_report(sick_only_mult_model_silver, title, 'is_multiple', 
                                  B=B, random_seed=random_seed)

### Multiple: Random Forest

In [ ]:
best_mult_rf_biased.fit(biased_text, biased_mult, rf__sample_weight=biased_weights)
best_mult_rf_gold.fit(gold_text, gold_mult, rf__sample_weight=gold_weights)
best_mult_rf_silver.fit(silver_text, silver_mult, rf__sample_weight=silver_weights)
print()

In [ ]:
start = time.time()
title = 'Random Forest Mult Biased'
all_results[title] = model_report(best_mult_rf_biased, title, 'is_multiple',
                                  save_fname='figures/mult_rf_biased',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'Random Forest Mult Gold'
all_results[title] = model_report(best_mult_rf_gold, title, 'is_multiple',
                                  save_fname='figures/mult_rf_gold',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'Random Forest Mult Silver'
all_results[title] = model_report(best_mult_rf_silver, title, 'is_multiple',
                                  save_fname='figures/mult_rf_silver',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
print_model_hyperparams(best_mult_rf_biased, 'Random Forest Mult Biased')
print()
print_model_hyperparams(best_mult_rf_gold, 'Random Forest Mult Gold')
print()
print_model_hyperparams(best_mult_rf_silver, 'Random Forest Mult Silver')
print()

### Multiple: SVM

In [ ]:
best_mult_svm_biased.fit(biased_text, biased_mult, svc__sample_weight=biased_weights)
best_mult_svm_gold.fit(gold_text, gold_mult, svc__sample_weight=gold_weights)
best_mult_svm_silver.fit(silver_text, silver_mult, svc__sample_weight=silver_weights)
print()

In [ ]:
start = time.time()
title = 'SVM Mult Biased'
all_results[title] = model_report(best_mult_svm_biased, title, 'is_multiple',
                                  save_fname='figures/mult_svm_biased',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'SVM Mult Gold'
all_results[title] = model_report(best_mult_svm_gold, title, 'is_multiple',
                                  save_fname='figures/mult_svm_gold',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
start = time.time()
title = 'SVM Mult Silver'
all_results[title] = model_report(best_mult_svm_silver, title, 'is_multiple',
                                  save_fname='figures/mult_svm_silver',
                                  test_data=test_data,
                                  B=B, random_seed=random_seed)
print('\n{} seconds'.format(int(time.time()-start)))

In [ ]:
print_model_hyperparams(best_mult_svm_biased, 'SVM Mult Biased')
print()
print_model_hyperparams(best_mult_svm_gold, 'SVM Mult Gold')
print()
print_model_hyperparams(best_mult_svm_silver, 'SVM Mult Silver')
print()

### Multiple: Prototype

The prototype classifier is build in the Weka library in Java. To evaluate this, we ran the model on the same test data in the java environment and output the scores to csv.

In [ ]:
prototype_mult_scores = pd.read_csv('data/prototype_mult_scores.csv')
prototype_mult_scores.head()

In [ ]:
prototype_mult_results = prototype_model_report(test_data['is_multiple'], 
                                    prototype_mult_scores['0'].tolist(), 
                                    test_data['is_biased'], 'J4.8', 
                                    B=B, random_seed=random_seed)
all_results['Prototype Multiple'] = prototype_mult_results

## Compile the scores into a nice table

In [ ]:
sick_table = pd.DataFrame()
mult_table = pd.DataFrame()
for name, result in sorted(all_results.items(), key=lambda x:x[0]):
    data = {k:v for k,v in result.items() if ('samples' not in k) and ('_ci' not in k) }
    data.update({k+'_b':v[0] for k,v in result.items() if '_ci' in k})
    data.update({k+'_t':v[1] for k,v in result.items() if '_ci' in k})
    data['name'] = name
    if 'Sick' in name:
        sick_table = sick_table.append(data, ignore_index=True)
    else:
        mult_table = mult_table.append(data, ignore_index=True)
sick_table.set_index('name', inplace=True)
mult_table.set_index('name', inplace=True)

In [ ]:
print(sick_table.columns)

In [ ]:
sick_table.to_csv('sick_results.csv')
sick_table[['mixed_f1', 'mixed_f1_ci_b', 'mixed_f1_ci_t']]

In [ ]:
mult_table.to_csv('mult_results.csv')
mult_table[['mixed_f1', 'mixed_f1_ci_b', 'mixed_f1_ci_t']]

### Write out the final Sick and Multiple classifiers -- Sick LR Silver and Mult LR Pipelined Silver

In [ ]:
joblib.dump(best_sick_lr_silver, 'final_models/final_yelp_sick_model.gz')
joblib.dump(sick_only_mult_lr_silver, 'final_models/final_yelp_mult_model.gz')